## Notebook setup

In [ ]:
import os
import io
import warnings
import datetime
import numpy as np
import base64

import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline

# import sys
# sys.path.append('../../')

from openbb_terminal import api as openbb
from openbb_terminal.helper_classes import TerminalStyle

%matplotlib inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")
warnings.filterwarnings("ignore")

# Detect if prediction capabilities are present. If they are not, disable prediction in the rest of the script
# so that the report can still be generated without prediction results.
predictions = True
try:
    openbb.stocks.pred.models
except Exception as e:
    predictions = False


In [ ]:
try:
    theme = TerminalStyle("light", "light", "light")
except:
    pass
stylesheet = openbb.widgets.html_report_stylesheet()
with open("./openbb_terminal/reports/OpenBB_reports_logo.png", "rb") as image_file:
    openbb_image_encoded = base64.b64encode(image_file.read())

## Select Ticker

In [ ]:
# Parameters that will be replaced when calling this notebook
ticker = "TSLA"
report_name = ""

In [ ]:
if "." in ticker:
    import sys
    sys.exit(0)

In [ ]:
ticker_data = openbb.stocks.load(ticker, start=datetime.datetime.now() - datetime.timedelta(days=4*30))
ticker_data = openbb.stocks.process_candle(ticker_data)

author = "Didier Rodrigues Lopes"
report_title = f"INVESTMENT RESEARCH REPORT ON {ticker.upper()}"
report_date =  datetime.datetime.now().strftime('%d %B, %Y')
report_time =  datetime.datetime.now().strftime('%H:%M') 
report_timezone = datetime.datetime.now(datetime.timezone.utc).astimezone().tzinfo
report_title, report_date, report_time, report_timezone

In [ ]:
info = openbb.stocks.fa.models.yahoo_finance.get_info(ticker=ticker).transpose()

if info["Long business summary"][0] != "NA":
    overview = info["Long business summary"][0]
else:
    overview = info["Long name"][0]
overview

## Data

In [ ]:
(
    df_year_estimates,
    df_quarter_earnings,
    df_quarter_revenues,
) = openbb.stocks.dd.models.business_insider.get_estimates(ticker)

In [ ]:
display_year = sorted(df_year_estimates.columns.tolist())[:3]
df_year_estimates = df_year_estimates[display_year].head(5)
df_year_estimates

In [ ]:
df_quarter_earnings

In [ ]:
df_quarter_revenues

In [ ]:
df_sec_filings = openbb.stocks.dd.models.marketwatch.get_sec_filings(ticker)[
    ["Type", "Category", "Link"]
].head(5)
df_sec_filings

In [ ]:
df_analyst = openbb.stocks.dd.models.finviz.get_analyst_data(ticker)
df_analyst["target_to"] = df_analyst["target_to"].combine_first(df_analyst["target"])
df_analyst = df_analyst[["category", "analyst", "rating", "target_to"]].rename(
    columns={
        "category": "Category",
        "analyst": "Analyst",
        "rating": "Rating",
        "target_to": "Price Target",
    }
)
df_analyst

In [ ]:
fig, ax1 = plt.subplots(figsize=(11, 5), dpi=150)
ax2 = ax1.twinx()
openbb.stocks.dps.spos(
    ticker,
    num=84,
    raw=False,
    export="",
    external_axes=[ax1, ax2],
)
fig.tight_layout()

f = io.BytesIO()
fig.savefig(f, format="svg")
net_short_position = f.getvalue().decode("utf-8")

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.stocks.dps.dpotc(
    ticker,
    external_axes=[ax1, ax2],
)
fig.tight_layout()

f = io.BytesIO()
fig.savefig(f, format="svg")
dark_pools = f.getvalue().decode("utf-8")

In [ ]:
fig, ax = plt.subplots(figsize=(11, 5), dpi=150)
openbb.stocks.dps.volexch(
    ticker,
    external_axes=[ax],
)
fig.tight_layout()

f = io.BytesIO()
fig.savefig(f, format="svg")
volexch = f.getvalue().decode("utf-8")

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
ax3 = ax1.twinx()
openbb.stocks.dps.psi_sg(
    ticker,
    external_axes=[ax1, ax2, ax3],
)
fig.tight_layout()

f = io.BytesIO()
fig.savefig(f, format="svg")
price_vs_short_interest = f.getvalue().decode("utf-8")

In [ ]:
fig, (candles, volume) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.stocks.candle(
    s_ticker=ticker,
    df_stock=ticker_data,
    use_matplotlib=True,
    external_axes=[candles, volume],
)
candles.set_xticklabels("")
fig.tight_layout()

f = io.BytesIO()
fig.savefig(f, format="svg")
price_chart = f.getvalue().decode("utf-8")

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.stocks.dd.pt(
    ticker=ticker,
    start="2022-01-01",
    interval="1440min",
    stock=ticker_data,
    num=10,
    raw=False,
    external_axes=[ax],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
price_target_chart = f.getvalue().decode("utf-8")

In [ ]:
df = openbb.stocks.dd.models.business_insider.get_price_target_from_analysts(ticker)
avg_ratings_last_30_days = round(np.mean(df[datetime.datetime.now() - datetime.timedelta(days=30):]["Price Target"].values), 2)
last_price = round(ticker_data["Close"][-1], 2)

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.stocks.dd.rot(
    ticker=ticker,
    num=10,
    raw=False,
    export="",
    external_axes=[ax],
)
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
ratings_over_time_chart = f.getvalue().decode("utf-8")

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 3), dpi=150)
openbb.stocks.ta.rsi(ticker_data["Close"], external_axes=[ax1, ax2])
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
ta_rsi = f.getvalue().decode("utf-8")

In [ ]:
df = openbb.stocks.ta.models.momentum.rsi(ticker_data["Close"])
rsi_value = round(df.values[-1][0], 2)
rsi_value

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(np.array(range(len(ticker_data["Close"][-30:].index))).reshape(-1,1), ticker_data["Close"][-30:].values)
regression_slope = round(model.coef_[0], 2)

In [ ]:
import pandas as pd
df_insider = pd.DataFrame.from_dict(openbb.stocks.ins.models.finviz.get_last_insider_activity(ticker)).head(10)
df_insider["Val ($)"] = df_insider["Value ($)"].replace({',': ''}, regex=True)
df_insider["Trade"] = df_insider.apply(
        lambda row: (-1*float(row["Val ($)"])) if row["Transaction"] == "Sale" else (float(row["Val ($)"]) if row["Transaction"] == "Buy" else 0),
        axis=1,
    )
last_10_insider_trading = round(sum(df_insider["Trade"])/1_000_000, 2)
df_insider = df_insider.drop(columns=["Val ($)", "Trade"])
df_insider

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.stocks.ba.headlines(ticker, external_axes=[ax])
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
finbrain_sentiment = f.getvalue().decode("utf-8")

In [ ]:
df_sentiment_finbrain = openbb.stocks.ba.models.finbrain.get_sentiment(ticker)
finbrain_sentiment_val = float(df_sentiment_finbrain.values[-1][0])

In [ ]:
watchlist_count, n_cases, n_bull, n_bear = openbb.stocks.ba.models.stocktwits.get_bullbear(ticker)
stocktwits_sentiment = f"Watchlist count: {watchlist_count}</br>"
if n_cases > 0:
    stocktwits_sentiment += f"\nLast {n_cases} sentiment messages:</br>"
    stocktwits_sentiment += f"Bullish {round(100*n_bull/n_cases, 2)}%</br>"
    stocktwits_sentiment += f"Bearish {round(100*n_bear/n_cases, 2)}%"
else:
    stocktwits_sentiment += "No messages found"

In [ ]:
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(11, 5), dpi=150)
openbb.stocks.ba.snews(ticker, external_axes=[ax1, ax2])
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
snews = f.getvalue().decode("utf-8")

In [ ]:
ticker_data_all = openbb.stocks.load(ticker, start=datetime.datetime.now() - datetime.timedelta(days=5*12*21))
ticker_data_all["Returns"] = ticker_data_all["Adj Close"].pct_change()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.stocks.qa.bw(ticker, ticker_data_all, "Returns", False, external_axes=[ax])
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
bw_month = f.getvalue().decode("utf-8")

In [ ]:
fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
openbb.stocks.qa.bw(ticker, ticker_data_all, "Returns", True, external_axes=[ax])
fig.tight_layout()
f = io.BytesIO()
fig.savefig(f, format="svg")
bw_year = f.getvalue().decode("utf-8")

In [ ]:
if predictions:
    regression_val = round(openbb.stocks.pred.models.regression.get_regression_model(ticker_data_all["Close"], 1, 80, 20, 1)[0][-1], 2)
    regression_val

In [ ]:
if predictions:
    fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
    openbb.stocks.pred.regression(ticker, ticker_data_all["Close"], 1, 80, 20, 1, external_axes=[ax])
    fig.tight_layout()
    f = io.BytesIO()
    fig.savefig(f, format="svg")
    regression = f.getvalue().decode("utf-8")

In [ ]:
if predictions:
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(11, 3), dpi=150)
    openbb.stocks.pred.mc(ticker_data["Close"], 30, 100, external_axes=[ax1, ax2])
    fig.tight_layout()
    f = io.BytesIO()
    fig.savefig(f, format="svg")
    mc = f.getvalue().decode("utf-8")

In [ ]:
if predictions:
    fig, ax = plt.subplots(figsize=(11, 3), dpi=150)
    openbb.stocks.pred.regression(ticker, ticker_data_all["Close"], 1, 80, 20, 1, external_axes=[ax])
    fig.tight_layout()
    f = io.BytesIO()
    fig.savefig(f, format="svg")
    regression = f.getvalue().decode("utf-8")

## Render the report template to a file

In [ ]:
body = ""

img = f'<img src="data:image/png;base64,{openbb_image_encoded.decode()}" alt="OpenBB" style="width:144px;">'
body += openbb.widgets.header(img, author, report_date, report_time, report_timezone, f'<b>INVESTMENT RESEARCH REPORT:</b> {ticker}')

body += openbb.widgets.tablinks(["SUMMARY", "Overview", "Analyst Opinions", "Fundamental Analysis", "Dark Pool and Shorts", "Technical Analysis", "Insider Trading", "Behavioural Analysis", "Quantitative Analysis", "Prediction Techniques"])

htmlcode = openbb.widgets.h(3, "KPIs")
htmlcode += openbb.widgets.kpi([last_price], ["Last closing price is above the average price ratings of last 30 days", "Average price ratings of last 30 day is above last closing price"], avg_ratings_last_30_days)
if predictions:
    htmlcode += openbb.widgets.kpi([0], ["Regression (dollars per market day) on last 30 market days is negative", "Regression (dollars per market day) on last 30 market days is positive"], regression_slope)
htmlcode += openbb.widgets.kpi([30, 70], ["RSI level is oversold", "RSI level is normal", "RSI level is overbought"], rsi_value) 
htmlcode += openbb.widgets.kpi([0], ["The sum of last 10 insider trading (in millions) was negative", "The sum of last 10 insider trading (in millions) was positive"], last_10_insider_trading)
htmlcode += openbb.widgets.kpi([-.1, .1], ["Last FinBrain sentiment is bearish", " Last FinBrain sentiment is neutral", "Last FinBrain sentiment is bullish"], finbrain_sentiment_val) 
if predictions:
    htmlcode += openbb.widgets.kpi([0], ["The regression for the next 20th market price is below closing price", "The regression for the next 20th market price is above closing price"], round(regression_val - last_price, 2)) 
body += openbb.widgets.add_tab("SUMMARY", htmlcode)

htmlcode = openbb.widgets.row([price_chart])
htmlcode += openbb.widgets.row([openbb.widgets.h(3, "Description") + openbb.widgets.p(overview)])
body += openbb.widgets.add_tab("Overview", htmlcode)

htmlcode = openbb.widgets.row([price_target_chart])
htmlcode += openbb.widgets.row([ratings_over_time_chart])
htmlcode += openbb.widgets.row([df_analyst.to_html()])
body += openbb.widgets.add_tab("Analyst Opinions", htmlcode)

htmlcode = openbb.widgets.row([openbb.widgets.h(3, "Estimates") + df_year_estimates.head().to_html()])
htmlcode += openbb.widgets.row([openbb.widgets.h(3, "Earnings") + df_quarter_earnings.head().to_html()])
htmlcode += openbb.widgets.row([openbb.widgets.h(3, "Revenues") + df_quarter_revenues.head().to_html()])
htmlcode += openbb.widgets.row([openbb.widgets.h(3, "SEC filings") + df_sec_filings.head().to_html()])
body += openbb.widgets.add_tab("Fundamental Analysis", htmlcode)
            
htmlcode = openbb.widgets.row([net_short_position])
htmlcode += openbb.widgets.row([price_vs_short_interest])
htmlcode += openbb.widgets.row([dark_pools])
htmlcode += openbb.widgets.row([volexch])
body += openbb.widgets.add_tab("Dark Pool and Shorts", htmlcode)

body += openbb.widgets.add_tab("Technical Analysis", openbb.widgets.row([ta_rsi]))

htmlcode = openbb.widgets.row([openbb.widgets.h(3, "Last Activity") + df_insider.head(10).to_html(col_space='75px')])
body += openbb.widgets.add_tab("Insider Trading", htmlcode)

htmlcode = openbb.widgets.row([finbrain_sentiment])
htmlcode += openbb.widgets.row([snews])
htmlcode += openbb.widgets.row([openbb.widgets.h(3, "Stocktwits") + stocktwits_sentiment])
body += openbb.widgets.add_tab("Behavioural Analysis", htmlcode)

htmlcode = openbb.widgets.row([bw_month])
htmlcode += openbb.widgets.row([bw_year])
body += openbb.widgets.add_tab("Quantitative Analysis", htmlcode)

if predictions:
    htmlcode = openbb.widgets.row([regression])
    htmlcode += openbb.widgets.row([mc])
else:
    htmlcode = openbb.widgets.row(["Prediction features not enabled."])
body += openbb.widgets.add_tab("Prediction Techniques", htmlcode)

body += openbb.widgets.tab_clickable_evt()

report = openbb.widgets.html_report(title=report_name, stylesheet=stylesheet, body=body)

# to save the results
with open(report_name + ".html", "w", encoding='utf-8') as fh:
    fh.write(report)